In [1]:
import spotipy as sp
from spotipy.oauth2 import SpotifyClientCredentials
from googlesearch import search
import requests
from bs4 import BeautifulSoup
import re
import string
import ids
import tensorflow as tf
import keras
from keras.preprocessing.text import Tokenizer
import numpy as np

In [2]:
spot_client_id=ids.SPOT_CLIENT_ID
spot_client_secret=ids.SPOT_CLIENT_SECRET
spot_client_credentials_manager = SpotifyClientCredentials(client_id=spot_client_id, client_secret=spot_client_secret)
spotify = sp.Spotify(client_credentials_manager=spot_client_credentials_manager)

genius_access_token = ids.GENIUS_ACCESS_TOKEN
genius_header = {'Authorization': f'Bearer {genius_access_token}'}

In [3]:
artist = 'rex orange county'
artist_search = spotify.search(q=f'artist:{artist}', type='artist')
artist_uri = artist_search['artists']['items'][0]['uri']

In [23]:
tokenizer = Tokenizer()

data = open('/tmp/irish-lyrics-eof.txt').read()

corpus = data.lower().split("\n")

tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1

print(tokenizer.word_index)
print(total_words)


{'the': 1, 'and': 2, 'i': 3, 'to': 4, 'a': 5, 'of': 6, 'my': 7, 'in': 8, 'me': 9, 'for': 10, 'you': 11, 'all': 12, 'was': 13, 'she': 14, 'that': 15, 'on': 16, 'with': 17, 'her': 18, 'but': 19, 'as': 20, 'when': 21, 'love': 22, 'is': 23, 'your': 24, 'it': 25, 'will': 26, 'from': 27, 'by': 28, 'they': 29, 'be': 30, 'are': 31, 'so': 32, 'he': 33, 'old': 34, 'no': 35, 'oh': 36, 'ill': 37, 'at': 38, 'one': 39, 'his': 40, 'there': 41, 'were': 42, 'heart': 43, 'down': 44, 'now': 45, 'we': 46, 'where': 47, 'young': 48, 'never': 49, 'go': 50, 'come': 51, 'then': 52, 'did': 53, 'not': 54, 'said': 55, 'away': 56, 'their': 57, 'sweet': 58, 'them': 59, 'green': 60, 'if': 61, 'take': 62, 'our': 63, 'like': 64, 'night': 65, 'day': 66, 'o': 67, 'out': 68, 'fair': 69, 'this': 70, 'town': 71, 'have': 72, 'can': 73, 'true': 74, 'its': 75, 'thou': 76, 'see': 77, 'dear': 78, 'more': 79, 'theres': 80, 'or': 81, 'had': 82, 'would': 83, 'over': 84, 'hear': 85, 'up': 86, 'ive': 87, 'through': 88, 'home': 89, '

In [24]:
song_list = []
for i in range(10):
    song_list.append(spotify.artist_top_tracks(artist_uri)['tracks'][i]['name'])

song_list

['THE SHADE',
 'Pluto Projector',
 'Best Friend',
 'AMAZING',
 'Television / So Far So Good',
 'Sunflower',
 'Loving Is Easy',
 'Happiness',
 'Corduroy Dreams',
 'Untitled']

In [5]:
word_arr = []
lyric_arr = []

for song in song_list:
    query = f'http://api.genius.com/search?q={artist}-{song}'

    search_req = requests.get(query, headers=genius_header)
    url = search_req.json()['response']['hits'][0]['result']['url']

    result = requests.get(url)
    content = result.content
    soup = BeautifulSoup(content, 'html.parser')
    result.close()

    lyrics = ''

    for tag in soup.select('div[class^="Lyrics__Container"], .song_body-lyrics p'):
        t = tag.get_text(strip=True, separator='\n')

        if t:
            lyrics += t

    lyrics = re.sub('\\[.*?\\]', '', lyrics)
    lyrics = re.sub('\n\n', '\n', lyrics)

    words = []
    for word in lyrics.split(' '):
        if not '\n' in word:
            words.append(word)
        else:
            phrase = word.split('\n')
            for i in range(len(phrase)-1):
                if phrase[i] != '':
                    words.append(phrase[i])
                    words.append('\n')
            if phrase[-1] != '':
                words.append(phrase[-1])     

    word_arr.append(words) 

    lyric_arr.append(lyrics)

word_arr

[['(Two,',
  'three,',
  'four)',
  '\n',
  'I',
  'was',
  'closing',
  'all',
  'the',
  'blinds',
  '\n',
  'Just',
  'so',
  'you',
  'could',
  'sleep',
  'the',
  'night',
  'through',
  '\n',
  'I',
  'was',
  'staying',
  'by',
  'your',
  'side',
  '\n',
  'Just',
  'so',
  'I',
  'knew',
  'you',
  'were',
  'okay',
  '\n',
  'And',
  'I',
  'was',
  'opening',
  'the',
  'door',
  '\n',
  'Just',
  'to',
  'see',
  'if',
  'you',
  'would',
  'walk',
  'through',
  '\n',
  'I',
  'started',
  'painting',
  'all',
  'the',
  'walls',
  '\n',
  'Just',
  'to',
  'see',
  'if',
  'you',
  'liked',
  'the',
  'shade',
  '\n',
  'I',
  'would',
  'love',
  'just',
  'to',
  'be',
  'stuck',
  'to',
  'your',
  'side',
  '\n',
  'Not',
  'with',
  'anybody',
  'else,',
  'anybody',
  'else',
  '\n',
  "It's",
  'enough',
  'just',
  'to',
  'keep',
  'us',
  'occupied',
  '\n',
  'Please',
  "don't",
  'go',
  '\n',
  'I',
  'would',
  'love',
  'just',
  'to',
  'be',
  'stuck',


In [6]:
all_songs = ''.join(lyric_arr)
print(all_songs[0:-1])


(Two, three, four)
I was closing all the blinds
Just so you could sleep the night through
I was staying by your side
Just so I knew you were okay
And I was opening the door
Just to see if you would walk through
I started painting all the walls
Just to see if you liked the shade
I would love just to be stuck to your side
Not with anybody else, anybody else
It's enough just to keep us occupied
Please don't go
I would love just to be stuck to your side
Not with anybody else, anybody else (Don't go)
It's enough just to keep us occupied (Please don't go)
You were holding out your hands
Hoping I'd be there to hold to
I went out to Amsterdam
Just so I could givе you some space
But I kept opеning my door
Just to see if you would walk through
But now I'm painting all my walls
Just to see if you like the shade
I would love just to be stuck to your side
Not with anybody else, anybody else
It's enough just to keep us occupied
Please don't go
I would love just to be stuck to your side
Not with any

In [7]:
# corpus = all_songs.lower().split('\n')[1:]
corpus = lyric_arr[0].lower().split('\n')[1:]

In [8]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)

In [9]:
total_words = len(tokenizer.word_index) + 1

In [10]:
input_sequences = []

for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0]
    
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

In [11]:
input_sequences

[[40, 41],
 [40, 41, 42],
 [3, 26],
 [3, 26, 43],
 [3, 26, 43, 27],
 [3, 26, 43, 27, 9],
 [3, 26, 43, 27, 9, 44],
 [2, 28],
 [2, 28, 6],
 [2, 28, 6, 30],
 [2, 28, 6, 30, 45],
 [2, 28, 6, 30, 45, 9],
 [2, 28, 6, 30, 45, 9, 46],
 [2, 28, 6, 30, 45, 9, 46, 29],
 [3, 26],
 [3, 26, 47],
 [3, 26, 47, 48],
 [3, 26, 47, 48, 10],
 [3, 26, 47, 48, 10, 15],
 [2, 28],
 [2, 28, 3],
 [2, 28, 3, 49],
 [2, 28, 3, 49, 6],
 [2, 28, 3, 49, 6, 31],
 [2, 28, 3, 49, 6, 31, 50],
 [51, 3],
 [51, 3, 26],
 [51, 3, 26, 52],
 [51, 3, 26, 52, 9],
 [51, 3, 26, 52, 9, 32],
 [2, 1],
 [2, 1, 17],
 [2, 1, 17, 18],
 [2, 1, 17, 18, 6],
 [2, 1, 17, 18, 6, 11],
 [2, 1, 17, 18, 6, 11, 33],
 [2, 1, 17, 18, 6, 11, 33, 29],
 [3, 53],
 [3, 53, 34],
 [3, 53, 34, 27],
 [3, 53, 34, 27, 9],
 [3, 53, 34, 27, 9, 35],
 [2, 1],
 [2, 1, 17],
 [2, 1, 17, 18],
 [2, 1, 17, 18, 6],
 [2, 1, 17, 18, 6, 54],
 [2, 1, 17, 18, 6, 54, 9],
 [2, 1, 17, 18, 6, 54, 9, 36],
 [3, 11],
 [3, 11, 19],
 [3, 11, 19, 2],
 [3, 11, 19, 2, 1],
 [3, 11, 19, 2, 1,

In [12]:
max_sequence_len = max([len(x) for x in input_sequences])

In [13]:
input_sequences = np.array(keras.utils.pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

In [14]:
xs = input_sequences[:,:-1]
labels = input_sequences[:,-1]

In [15]:
ys = keras.utils.to_categorical(labels, num_classes=total_words)

In [16]:
model = keras.Sequential([
    keras.layers.Embedding(total_words, 256, input_length=max_sequence_len-1),
    keras.layers.LSTM(1024),
    keras.layers.Dense(total_words, activation='softmax'),
])

In [17]:
model.compile(optimizer=keras.optimizers.Adam(lr=0.01), loss='categorical_crossentropy', metrics=['accuracy'])

In [18]:
history=model.fit(xs, ys, epochs=100, verbose=1)

Epoch 1/100
7/7 [==============================] - 1s 65ms/step - loss: 4.1766 - accuracy: 0.0829
Epoch 2/100
7/7 [==============================] - 0s 54ms/step - loss: 3.8592 - accuracy: 0.0976
Epoch 3/100
7/7 [==============================] - 0s 61ms/step - loss: 3.6846 - accuracy: 0.1561
Epoch 4/100
7/7 [==============================] - 0s 63ms/step - loss: 3.4456 - accuracy: 0.1366
Epoch 5/100
7/7 [==============================] - 0s 58ms/step - loss: 3.2310 - accuracy: 0.1756
Epoch 6/100
7/7 [==============================] - 0s 54ms/step - loss: 3.0625 - accuracy: 0.2341
Epoch 7/100
7/7 [==============================] - 0s 58ms/step - loss: 2.7318 - accuracy: 0.2878
Epoch 8/100
7/7 [==============================] - 0s 60ms/step - loss: 2.4126 - accuracy: 0.3512
Epoch 9/100
7/7 [==============================] - 0s 64ms/step - loss: 2.1637 - accuracy: 0.4390
Epoch 10/100
7/7 [==============================] - 0s 59ms/step - loss: 1.7733 - accuracy: 0.5122
Epoch 11/100
7/7 [=

In [19]:
corpus[-1]

"please don't go"

In [20]:
seed_text = "this could be"
next_words = 100
  
for i in range(next_words):
	token_list = tokenizer.texts_to_sequences([seed_text])[0]
	token_list = keras.utils.pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
	predicted = np.argmax(model.predict(token_list, verbose=0), axis=-1)
	output_word = ""
	for word, index in tokenizer.word_index.items():
		if index == predicted:
			output_word = word
			break
	seed_text += " " + output_word
	if i % 10 == 0:
		seed_text += '\n'
print(seed_text)

this could be out
 to amsterdam hold to side space space through through through
 through through through space space space space space space space
 space space space space space space space space space space
 space space space space space space space space space space
 space space space space space space space space space space
 space space space space space space space space space space
 space space space space space space space space space space
 space space space space space space space space space space
 space space space space space space space space space space
 space space space space space space space space space
